# ZBP to ZCTA
Retrieves data from the Census Bureau's ZIP Code Business Patterns API for a specific area and summarizes it by ZCTA. Three tables are generated: one for employees that contains employment, establishments, and wages, one for industries that contains counts of establishments by 2-digit sector NAICS codes, and one reference table that correlates sector numbers and names. Initial data retrieved from API is written to json, final output is written to a SQLite database. 

Confirmed to work with the 2021 ZBP series

https://www.census.gov/data/developers/data-sets/cbp-zbp/cbp-api.html

## Variables

In [1]:
import pandas as pd, requests, sqlite3, os, json
from IPython.display import clear_output

In [2]:
#Crosswalk file from UDS Mapper
uszips_file='ZIPCodetoZCTACrosswalk2022UDS.xlsx'
state_abrev='RI' #State with ZIPs we want to pull

uszips_path=os.path.join('inputs',uszips_file)

#Dump files for api data storage
ejsonpath=os.path.join('outputs', 'emp_data.json')
ijsonpath=os.path.join('outputs', 'ind_data.json')
cjsonpath=os.path.join('outputs', 'codes_data.json')

#API variables - UPDATE THE YEAR
keyfile='census_key.txt'

year='2021'
dsource='cbp'
state='44'
ecols='ESTAB,EMP,PAYQTR1,PAYANN'
icols='ESTAB'
ncodes=['00','11','21','22','23','31-33','42','44-45','48-49',
        '51','52','53','54','55','56','61','62','71','72','81',
        '99']

#SQL output - UPDATE EACH TABLE NAME
dbname=os.path.join('outputs','testdb.sqlite')
emptable='zbp2021_emp'
indtable='zbp2021_ind'
codetable='zbp2021_indcodes'
cwalktable='zip2zcta_2022'

## Crosswalking
Read in the file that relates US ZIP codes to ZCTAs (from UDS Mapper) and
pull out codes for specific state

In [3]:
uszips=pd.read_excel(uszips_path,dtype={'ZIP_CODE': str, 'zcta': str})
uszips.head()

,ZIP_CODE,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
0,00501,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
1,00544,Holtsville,NY,Post Office or large volume customer,11742,Spatial join to ZCTA
2,00601,Adjuntas,PR,Zip Code Area,00601,Zip matches ZCTA
3,00602,Aguada,PR,Zip Code Area,00602,Zip matches ZCTA
4,00603,Aguadilla,PR,Zip Code Area,00603,Zip matches ZCTA


In [4]:
#All ZIP Codes in US
uszips.shape

(41061, 6)

In [5]:
statezips=uszips.loc[uszips['STATE']==state_abrev].set_index('ZIP_CODE')
statezips.head()

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
ZIP_CODE,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA
02802,Albion,RI,Zip Code Area,02802,Zip matches ZCTA
02804,Ashaway,RI,Zip Code Area,02804,Zip matches ZCTA
02806,Barrington,RI,Zip Code Area,02806,Zip matches ZCTA
02807,Block Island,RI,Zip Code Area,02807,Zip matches ZCTA


In [6]:
#ZIPS in local area
statezips.shape

(90, 5)

## API Call and Processing
Request the ZBP data from the Census Bureau for the state, join the ZBP data to the local ZCTA to ZIP table based on ZIP Code, and group the data by ZCTA 

In [7]:
with open(keyfile) as key:
    api_key=key.read().strip()

In [8]:
base_url = f'https://api.census.gov/data/{year}/{dsource}'
base_url

'https://api.census.gov/data/2021/cbp'

### ZBP Employment Data
This data is requested in a series of chunks which contain multiple ZIP Codes - do not rerun the requests block if retrieval is successful but subsequent notebook changes are needed. Proceed to the next block and pull data from json dump file.

In [9]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [10]:
reqzips=list(chunks(statezips.index.tolist(),48))
print('Number of chunks:',len(reqzips))

Number of chunks: 2


#### ***THIS BLOCK IS A REQUESTS BLOCK!*** 

In [11]:
#Code 200 = success, do not rerun this block unless it's necessary
emp_data=[]
for i, v in enumerate (reqzips):
    batchzips=','.join(v)
    edata_url = f'{base_url}?get={ecols}&EMPSZES=001&for=zip code:{batchzips}&key={api_key}'
    response=requests.get(edata_url)
    if response.status_code==200:
        clear_output(wait=True)
        data=response.json()
        if i == 0:    
            for record in data:
                emp_data.append(record)
        else:
            for record in data[1:]:
                emp_data.append(record) 
        print('Retrieved data for chunk',i)
    else:
        print('***Problem with retrieval***, response code',response.status_code)
        break
with open(ejsonpath, 'w') as f:
    json.dump(emp_data, f)
print('Done - Data dumped to json file')

Retrieved data for chunk 1
Done - Data dumped to json file


In [12]:
with open(ejsonpath, 'r') as f:
    ejsondata=json.load(f)
zbpemp=pd.DataFrame(ejsondata[1:], columns=ejsondata[0]).rename(columns={'PAYQTR1':'PAYQ1','PAYANN':'PAYAN','zip code':'zipcode'}).set_index('zipcode')
for field in zbpemp.columns:
    zbpemp=zbpemp.astype(dtype={field:'int64'})
zbpemp.drop(columns=['EMPSZES'],inplace=True)
zbpemp.head()

,ESTAB,EMP,PAYQ1,PAYAN
zipcode,,,,
02801,5,24,681,2740
02802,9,61,342,2288
02804,79,522,5256,26600
02806,363,3467,29647,136816
02874,101,356,3347,20832


In [13]:
#ZIP Codes retrieved - may differ from statezips as some zips have no businesses
zbpemp.shape

(87, 4)

In [14]:
# Checking what those missing ZIPs are, for which there is no ZBP data
statezips.index.difference(zbpemp.index)

Index(['02823', '02836', '02877'], dtype='object')

In [15]:
#Flag columns count the number of establishments for which data is not disclosed
flags=['FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']
for flagcol in flags:
    datacol=flagcol.split('_')[1]
    zbpemp[flagcol]=0
    zbpemp.loc[zbpemp[datacol] == 0, flagcol] = zbpemp['ESTAB']
zbpemp.head()

,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
zipcode,,,,,,,
02801,5,24,681,2740,0,0,0
02802,9,61,342,2288,0,0,0
02804,79,522,5256,26600,0,0,0
02806,363,3467,29647,136816,0,0,0
02874,101,356,3347,20832,0,0,0


In [16]:
#Join to ZIP ZCTA crosswalk
zbpemp2zcta = pd.merge(statezips,zbpemp,how='inner',left_index=True,right_index=True)
zbpemp2zcta.index.name = 'ZIP_CODE'
zbpemp2zcta.head()

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
ZIP_CODE,,,,,,,,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA,5,24,681,2740,0,0,0
02802,Albion,RI,Zip Code Area,02802,Zip matches ZCTA,9,61,342,2288,0,0,0
02804,Ashaway,RI,Zip Code Area,02804,Zip matches ZCTA,79,522,5256,26600,0,0,0
02806,Barrington,RI,Zip Code Area,02806,Zip matches ZCTA,363,3467,29647,136816,0,0,0
02807,Block Island,RI,Zip Code Area,02807,Zip matches ZCTA,191,442,3839,40212,0,0,0


In [17]:
#ZIP count
zbpemp2zcta.shape

(87, 12)

In [18]:
#Aggregate to ZCTAs
zctaemp=zbpemp2zcta[['zcta','ESTAB','EMP','PAYQ1','PAYAN','FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']].groupby(['zcta'])[['ESTAB','EMP','PAYQ1','PAYAN','FLAG_EMP','FLAG_PAYQ1','FLAG_PAYAN']].sum()
zctaemp.head()

,ESTAB,EMP,PAYQ1,PAYAN,FLAG_EMP,FLAG_PAYQ1,FLAG_PAYAN
zcta,,,,,,,
02802,9,61,342,2288,0,0,0
02804,79,522,5256,26600,0,0,0
02806,363,3467,29647,136816,0,0,0
02807,191,442,3839,40212,0,0,0
02808,28,102,818,3814,0,0,0


In [19]:
# ZCTA count
# RI has 81 ZCTAs in 2020, but two 02823 and 02836 had no biz establishments in 2021 ZBP
zctaemp.shape

(79, 7)

In [20]:
#These are point-based ZIPs that were aggregated to a ZCTA
ptzips=list(zbpemp2zcta.index.difference(zctaemp.index))
zbpemp2zcta.index.difference(zctaemp.index)

Index(['02801', '02862', '02880', '02883', '02887', '02901', '02902', '02940'], dtype='object')

In [21]:
#Double check and make sure that's what these are
#Confirmed for RI that these are point-based and have no area
statezips.loc[statezips.index.isin(ptzips)]

,PO_NAME,STATE,ZIP_TYPE,zcta,zip_join_type
ZIP_CODE,,,,,
02801,Adamsville,RI,Post Office or large volume customer,02837,Spatial join to ZCTA
02862,Pawtucket,RI,Post Office or large volume customer,02860,Spatial join to ZCTA
02880,Wakefield,RI,Post Office or large volume customer,02879,Spatial join to ZCTA
02883,Peace Dale,RI,Post Office or large volume customer,02879,Spatial join to ZCTA
02887,Warwick,RI,Post Office or large volume customer,02886,Spatial join to ZCTA
02901,Providence,RI,Post Office or large volume customer,02903,Spatial join to ZCTA
02902,Providence,RI,Post Office or large volume customer,02903,Spatial join to ZCTA
02940,Providence,RI,Post Office or large volume customer,02904,Spatial join to ZCTA


### ZBP Industry Data
This data must be requested one record at a time. NOTE that this request can take a LONG TIME, up to 20 minutes for approx 100 ZIP Codes. Once the request is finished the data gets dumped into a json file. If the request is successful but subsequent blocks need to be modified, don't rerun the requests block - pull the data from the json file.

In [22]:
zipcodes=zbpemp2zcta.index.tolist()
len(zipcodes)

87

# ***THIS BLOCK IS A REQUESTS BLOCK!***  
Retrieving approx 100 ZIP Codes takes 20 minutes

*NOTE - revise in the future to retrieve chunks of zip codes*

In [23]:
#If this block is successful but there are subsequent problems, do not rerun it - start from the following block.
#For industry data, if there are no records for an industry create a blank record with zeros
n=0
z=0
ind_data=[['estab','naics','zipcode']]
for zcode in zipcodes:
#for zcode in zipcodes[0:5]:
    clear_output(wait=True)
    for naics in ncodes:
        idata_url = f'{base_url}?get={icols}&NAICS2017={naics}&for=zip code:{zcode}&key={api_key}'
        try:
            response=requests.get(idata_url)
        except requests.exceptions.RequestException as e:
            print (e)
            break
        if response.status_code==200:
            jsondata=response.json()
            ind_data.append(jsondata[1]) 
            n=n+1
        elif response.status_code==204:
            record=['0',naics,zcode]
            ind_data.append(record)
            n=n+1
        else:
            print('Problem retrieving data, status code:',response.status_code)
            break
    z=z+1
    print(n,'records have been retrieved for',z,'ZIP codes...')
print('Done')

with open(ijsonpath, 'w') as f:
    json.dump(ind_data, f)
print('Data dumped to json file')

1800 records have been retrieved for 87 ZIP codes...
Done
Data dumped to json file


In [24]:
with open(ijsonpath, 'r') as f:
    ijsondata=json.load(f)
zbpind = pd.DataFrame(ijsondata[1:],columns=ijsondata[0])
zbpind['estab']=zbpind['estab'].astype('int64')
zbpind.head()

,estab,naics,zipcode
0,5,00,02801
1,0,11,02801
2,0,21,02801
3,0,22,02801
4,0,23,02801


In [25]:
#Pivot data to move NAICS to columns
zbpind_tab=zbpind.pivot(index='zipcode', columns='naics', values='estab')
zbpind_tab=zbpind_tab.add_prefix('N')
zbpind_tab.rename(columns=lambda x: x.replace('-', '_'),inplace=True)
zbpind_tab.head()

naics,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N53,N54,N55,N56,N61,N62,N71,N72,N81,N99
zipcode,,,,,,,,,,,,,,,,,,,,,
02801,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
02804,79.0,0.0,0.0,0.0,22.0,3.0,4.0,9.0,0.0,0.0,...,0.0,8.0,0.0,8.0,0.0,4.0,0.0,7.0,8.0,0.0
02806,363.0,0.0,0.0,0.0,44.0,4.0,9.0,28.0,0.0,10.0,...,23.0,49.0,0.0,24.0,12.0,56.0,18.0,22.0,35.0,0.0
02807,191.0,0.0,0.0,0.0,31.0,5.0,0.0,33.0,6.0,0.0,...,11.0,3.0,0.0,17.0,0.0,0.0,6.0,59.0,12.0,0.0


In [26]:
#Create column to summarize businesses that were not disclosed and categorized, but that are included in the total
zbpind_tab['NXX']=zbpind_tab.loc[:,'N00'].subtract(zbpind_tab.loc[:,'N11':'N99'].sum(axis=1))
zbpind_tab.head()

naics,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
zipcode,,,,,,,,,,,,,,,,,,,,,
02801,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
02804,79.0,0.0,0.0,0.0,22.0,3.0,4.0,9.0,0.0,0.0,...,8.0,0.0,8.0,0.0,4.0,0.0,7.0,8.0,0.0,6.0
02806,363.0,0.0,0.0,0.0,44.0,4.0,9.0,28.0,0.0,10.0,...,49.0,0.0,24.0,12.0,56.0,18.0,22.0,35.0,0.0,2.0
02807,191.0,0.0,0.0,0.0,31.0,5.0,0.0,33.0,6.0,0.0,...,3.0,0.0,17.0,0.0,0.0,6.0,59.0,12.0,0.0,8.0


In [27]:
#Join to ZIP ZCTA crosswalk
zbpind2zcta = pd.merge(statezips[['zcta']],zbpind_tab,how='inner',left_index=True,right_index=True)
zbpind2zcta.index.name = 'ZIP_CODE'
zbpind2zcta.head()

,zcta,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
ZIP_CODE,,,,,,,,,,,,,,,,,,,,,
02801,02837,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
02802,02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
02804,02804,79.0,0.0,0.0,0.0,22.0,3.0,4.0,9.0,0.0,...,8.0,0.0,8.0,0.0,4.0,0.0,7.0,8.0,0.0,6.0
02806,02806,363.0,0.0,0.0,0.0,44.0,4.0,9.0,28.0,0.0,...,49.0,0.0,24.0,12.0,56.0,18.0,22.0,35.0,0.0,2.0
02807,02807,191.0,0.0,0.0,0.0,31.0,5.0,0.0,33.0,6.0,...,3.0,0.0,17.0,0.0,0.0,6.0,59.0,12.0,0.0,8.0


In [28]:
zbpind2zcta.shape

(87, 23)

In [29]:
#Aggregate to ZCTAs
zctaind=zbpind2zcta.groupby(['zcta']).sum()
zctaind.head()

,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54,N55,N56,N61,N62,N71,N72,N81,N99,NXX
zcta,,,,,,,,,,,,,,,,,,,,,
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
02804,79.0,0.0,0.0,0.0,22.0,3.0,4.0,9.0,0.0,0.0,...,8.0,0.0,8.0,0.0,4.0,0.0,7.0,8.0,0.0,6.0
02806,363.0,0.0,0.0,0.0,44.0,4.0,9.0,28.0,0.0,10.0,...,49.0,0.0,24.0,12.0,56.0,18.0,22.0,35.0,0.0,2.0
02807,191.0,0.0,0.0,0.0,31.0,5.0,0.0,33.0,6.0,0.0,...,3.0,0.0,17.0,0.0,0.0,6.0,59.0,12.0,0.0,8.0
02808,28.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,8.0,0.0,8.0


In [30]:
zctaind.shape

(79, 22)

In [31]:
#Generate and calculate percent total columns
ncols=list(zctaind)
for c in ncols[1:]:
    pct=c+'_PCT'
    zctaind[pct]=((zctaind[c]/zctaind['N00'])*100).round(2)
zctaind.head()

,N00,N11,N21,N22,N23,N31_33,N42,N44_45,N48_49,N51,...,N54_PCT,N55_PCT,N56_PCT,N61_PCT,N62_PCT,N71_PCT,N72_PCT,N81_PCT,N99_PCT,NXX_PCT
zcta,,,,,,,,,,,,,,,,,,,,,
02802,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,33.33,0.00,0.00,0.00,0.00,0.00,0.0,66.67
02804,79.0,0.0,0.0,0.0,22.0,3.0,4.0,9.0,0.0,0.0,...,10.13,0.0,10.13,0.00,5.06,0.00,8.86,10.13,0.0,7.59
02806,363.0,0.0,0.0,0.0,44.0,4.0,9.0,28.0,0.0,10.0,...,13.50,0.0,6.61,3.31,15.43,4.96,6.06,9.64,0.0,0.55
02807,191.0,0.0,0.0,0.0,31.0,5.0,0.0,33.0,6.0,0.0,...,1.57,0.0,8.90,0.00,0.00,3.14,30.89,6.28,0.0,4.19
02808,28.0,0.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,...,0.00,0.0,14.29,0.00,0.00,0.00,0.00,28.57,0.0,28.57


### NAICS Codes

#### ***THIS BLOCK IS A REQUESTS BLOCK!***

In [32]:
codedict={}
codes_url=f'https://api.census.gov/data/2021/cbp/variables/NAICS2017.json'
response=requests.get(codes_url)
codes_data=response.json()
codedict.update(codes_data['values']['item'])
sectordict=dict((k, codedict[k]) for k in ncodes)
sectordict['XX']='Establishments omitted from classification due to privacy regulations'

In [33]:
codes=pd.DataFrame(list(sectordict.items()), columns=['naics', 'name']).set_index('naics')
codes

,name
naics,
00,Total for all sectors
11,"Agriculture, forestry, fishing and hunting"
21,"Mining, quarrying, and oil and gas extraction"
22,Utilities
23,Construction
31-33,Manufacturing
42,Wholesale trade
44-45,Retail trade
48-49,Transportation and warehousing


In [34]:
# Remove key from memory
api_key=None

## Quality Control Checks

In [35]:
#Does sum of industries equal industry total?
indsum=zctaind['N00'].subtract(zctaind.iloc[:,1:22].sum(axis=1))
if indsum.sum()==0:
    print (True)
else:
    print(indsum.loc[indsum != 0])

True


In [36]:
#Is sum of percent totals approximately 100?
ptotal=zctaind.iloc[:,22:].sum(axis=1)
if ptotal.loc[(ptotal <= 99.05) | (ptotal >= 100.05)].empty:
    print(True)
else:
    print(ptotal.loc[(ptotal <= 99.05) | (ptotal >= 100.05)])


True


In [37]:
#Do number of ZCTAs in employment table match the industries table?
ecount=zctaemp.shape[0]
icount=zctaind.shape[0]
if ecount == icount:
    print (True)
else:
    print('Mistmatched count between employment',ecount, 'rows and industry',icount, 'rows')

True


In [38]:
#Does sum of estabslishments from employment table equal establishments in industries table?
estsum=zctaemp['ESTAB'].subtract(zctaind['N00'])
if estsum.sum()==0:
    print (True)
else:
    print(estsum.loc[estsum != 0])

True


## Write to Database 

In [39]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [40]:
#Employment table
cur.execute('DROP TABLE IF EXISTS {};'.format(emptable))
qcreate_emptab="""
CREATE TABLE {}(
zcta TEXT NOT NULL PRIMARY KEY,
estab INTEGER,
emp INTEGER,
payq1 INTEGER,
payan INTEGER,
flag_emp INTEGER,
flag_payq1 INTEGER,
flag_payan INTEGER);
""".format(emptable)

cur.execute(qcreate_emptab)

In [41]:
#Don't run this block unless you've run the previous one
zctaemp.to_sql(name='{}'.format(emptable), if_exists='append', index=True, con=con)

In [42]:
cur.execute('SELECT COUNT(*) FROM {};'.format(emptable))
rows = cur.fetchone()
print(rows[0], 'records written to', emptable)

79 records written to zbp2021_emp


In [43]:
#Replace zeros with nulls, as these values really represent no data
for col in zctaemp.columns[1:]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0;'.format(emptable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

0 records updated for EMP
0 records updated for PAYQ1
0 records updated for PAYAN
79 records updated for FLAG_EMP
79 records updated for FLAG_PAYQ1
79 records updated for FLAG_PAYAN


In [44]:
#Industry table
cur.execute('DROP TABLE IF EXISTS {}'.format(indtable))
qcreate_indtab="""
CREATE TABLE {} (
zcta TEXT NOT NULL PRIMARY KEY, 
N00 INTEGER, 
N11 INTEGER, 
N21 INTEGER, 
N22 INTEGER, 
N23 INTEGER, 
N31_33 INTEGER, 
N42 INTEGER, 
N44_45 INTEGER, 
N48_49 INTEGER, 
N51 INTEGER, 
N52 INTEGER, 
N53 INTEGER, 
N54 INTEGER, 
N55 INTEGER, 
N56 INTEGER, 
N61 INTEGER, 
N62 INTEGER, 
N71 INTEGER, 
N72 INTEGER, 
N81 INTEGER, 
N99 INTEGER,
NXX INTEGER,
N11_PCT REAL, 
N21_PCT REAL, 
N22_PCT REAL, 
N23_PCT REAL, 
N31_33_PCT REAL, 
N42_PCT REAL, 
N44_45_PCT REAL, 
N48_49_PCT REAL, 
N51_PCT REAL, 
N52_PCT REAL, 
N53_PCT REAL, 
N54_PCT REAL, 
N55_PCT REAL, 
N56_PCT REAL, 
N61_PCT REAL, 
N62_PCT REAL, 
N71_PCT REAL, 
N72_PCT REAL, 
N81_PCT REAL, 
N99_PCT REAL,
NXX_PCT REAL);
""".format(indtable)

cur.execute(qcreate_indtab)

In [45]:
#Don't run this block unless you've run the previous one
zctaind.to_sql(name='{}'.format(indtable), if_exists='append', index=True, con=con)

In [46]:
cur.execute('SELECT COUNT(*) FROM {};'.format(indtable))
rows = cur.fetchone()
print(rows[0], 'records written to', indtable)

79 records written to zbp2021_ind


In [47]:
#For percentages, replace zeros with nulls, as these values really represent no data
for col in zctaind.columns[22:]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0.0;'.format(indtable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

76 records updated for N11_PCT
79 records updated for N21_PCT
75 records updated for N22_PCT
8 records updated for N23_PCT
26 records updated for N31_33_PCT
24 records updated for N42_PCT
17 records updated for N44_45_PCT
34 records updated for N48_49_PCT
43 records updated for N51_PCT
31 records updated for N52_PCT
24 records updated for N53_PCT
19 records updated for N54_PCT
60 records updated for N55_PCT
17 records updated for N56_PCT
36 records updated for N61_PCT
22 records updated for N62_PCT
31 records updated for N71_PCT
19 records updated for N72_PCT
18 records updated for N81_PCT
79 records updated for N99_PCT
3 records updated for NXX_PCT


In [48]:
#For establishments, replace zeros with nulls unless establishments were omitted from classification
for col in zctaind.columns[1:22]:
    qupdate='UPDATE {} SET {} = NULL WHERE {} = 0 AND NXX !=0;'.format(indtable,col,col)
    cur.execute(qupdate)
    print(cur.rowcount,'records updated for',col)
    con.commit()

73 records updated for N11
76 records updated for N21
73 records updated for N22
8 records updated for N23
25 records updated for N31_33
23 records updated for N42
16 records updated for N44_45
33 records updated for N48_49
42 records updated for N51
30 records updated for N52
23 records updated for N53
18 records updated for N54
59 records updated for N55
16 records updated for N56
35 records updated for N61
21 records updated for N62
30 records updated for N71
18 records updated for N72
17 records updated for N81
76 records updated for N99
0 records updated for NXX


In [49]:
#NAICS code table
cur.execute('DROP TABLE IF EXISTS {};'.format(codetable))
qcreate_codetab="""
CREATE TABLE {}(
naics TEXT NOT NULL PRIMARY KEY,
name TEXT);
""".format(codetable)

cur.execute(qcreate_codetab)

In [50]:
#Don't run this block unless you've run the previous one
codes.to_sql(name='{}'.format(codetable), if_exists='append', index=True, con=con)

In [51]:
cur.execute('SELECT COUNT(*) FROM {};'.format(codetable))
rows = cur.fetchone()
print(rows[0], 'records written to', codetable)

22 records written to zbp2021_indcodes


In [52]:
#Crosswalk table
cur.execute('DROP TABLE IF EXISTS {};'.format(cwalktable))
qcreate_z2ztab="""
CREATE TABLE {}(
ZIP_CODE TEXT NOT NULL PRIMARY KEY,
PO_NAME TEXT,
STATE TEXT,
ZIP_TYPE TEXT,
zcta TEXT,
zip_join_type TEXT);
""".format(cwalktable)

cur.execute(qcreate_z2ztab)

In [53]:
#Don't run this block unless you've run the previous one
statezips.to_sql(name='{}'.format(cwalktable), if_exists='append', index=True, con=con)

In [54]:
cur.execute('SELECT COUNT(*) FROM {};'.format(cwalktable))
rows = cur.fetchone()
print(rows[0], 'records written to', cwalktable)

90 records written to zip2zcta_2022


In [55]:
con.close()